In [1]:
!apt-get install -y mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 34 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich-dev amd64 4.0-3 [7,375 kB]
Fetched 14.2 MB in 

In [3]:
%%writefile hello_mpi.cpp
#include <iostream>
#include <omp.h>
using namespace std;

//Function to be integrated
double f(double x) {
    return x * x;  // Example: f(x) = x^2
}

int main() {
    double a, b;     // Integration limits
    int n;           // Number of trapezoids
    int num_threads; // Number of threads

    cout << "Enter lower limit (a): ";
    cin >> a;
    cout << "Enter upper limit (b): ";
    cin >> b;
    cout << "Enter number of trapezoids (n): ";
    cin >> n;
    cout << "Enter number of threads: ";
    cin >> num_threads;

    double h = (b - a) / n;  // Width of each trapezoid
    double integral = 0.0;

    double start_time = omp_get_wtime();

#pragma omp parallel num_threads(num_threads)
    {
        double local_sum = 0.0;

#pragma omp for
        for (int i = 1; i < n; i++) {
            double x = a + i * h;
            local_sum += 2.0 * f(x);
        }

#pragma omp critical
        {
            integral += local_sum;
        }
    }

    // Add the first and last terms
    integral += f(a) + f(b);
    integral *= (h / 2.0);

    double end_time = omp_get_wtime();

    cout << "Approximate value of the integral: " << integral << endl;
    cout << "Time taken: " << end_time - start_time << " seconds" << endl;

    return 0;
}

Writing hello_mpi.cpp


In [4]:
!g++ -fopenmp hello_mpi.cpp -o hello_mpi
!./hello_mpi

Enter lower limit (a): 3
Enter upper limit (b): 6
Enter number of trapezoids (n): 3
Enter number of threads: 2
Approximate value of the integral: 63.5
Time taken: 0.000194615 seconds
